# Raw data

# Convert raw data to standardized format

In [2]:
from pipeline.preprocessors import convert_to_standardized, spark_instance, RAW_DATA_DIR, STANDARDIZED_DATA_DIR

=== .:python-datatoolbox
{'dataset': 'learning_quality', 'input_files_dir': 'inputs', 'artifact_files_dir': 'artifacts', 'crucial_artifact_files_dir': 'crucial_artifacts', 'exportable_files_dir': 'exportable', 'tmp_dir': 'tmp', 'processing_pool_size': 10, 'customer_specific': {'agents': {'name': {'data_availability': {'agent_input': 'agent-name-inputs.csv', 'fields': {'alerted_party_field': 'ad_alerted_name', 'watchlist_field': 'ad_matched_name'}, 'crucial_artifact': 'agent-name-availability.csv'}, 'artifact_file': 'agent-name-results.csv', 'tuning_report_path': 'agent-name-tuning-report.csv'}, 'dob': {'data_availability': {'agent_input': 'agent-dob-inputs.csv', 'fields': {'alerted_party_field': 'ad_alerted_dob', 'watchlist_field': 'ad_matched_dob'}, 'crucial_artifact': 'agent-dob-availability.csv'}, 'artifact_file': 'agent-dob-results.csv'}, 'nationality': {'data_availability': {'agent_input': 'agent-nationality-inputs.csv', 'fields': {'alerted_party_field': 'ad_alerted_nationality', 

Ivy Default Cache set to: /app/dependencies/ivy2/cache
The jars for the packages stored in: /app/dependencies/ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-351ab865-3e15-4b31-b66d-53c0c8962b46;1.0
	confs: [default]


:: loading settings :: url = jar:file:/env/ds/anaconda/envs/pipeline/lib/python3.7/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
:: resolution report :: resolve 141ms :: artifacts dl 5ms
	:: modules in use:
	com.ibm.icu#icu4j;58.2 from central in [default]
	io.delta#delta-core_2.12;1.0.0 from central in [default]
	org.abego.treelayout#org.abego.treelayout.core;1.0.3 from central in [default]
	org.antlr#ST4;4.0.8 from central in [default]
	org.antlr#antlr-runtime;3.5.2 from central in [default]
	org.antlr#antlr4;4.7 from central in [default]
	org.antlr#antlr4-runtime;4.7 from central in [default]
	org.glassfish#javax.json;1.0.4 from central in [default]
	---------------------------------------------------------------------
	|             

21/12/10 12:02:39 INFO Executor: Starting executor ID driver on host 309651fbdaab
21/12/10 12:02:39 INFO Executor: Fetching file:///app/dependencies/ivy2/jars/org.abego.treelayout_org.abego.treelayout.core-1.0.3.jar with timestamp 1639137758791
21/12/10 12:02:39 INFO Utils: /app/dependencies/ivy2/jars/org.abego.treelayout_org.abego.treelayout.core-1.0.3.jar has been previously copied to /tmp/app/spark/spark-b4f75a4c-de12-4e0d-89fd-1bb64fc96015/userFiles-b97defce-65a3-4597-bd8d-92ff232a786e/org.abego.treelayout_org.abego.treelayout.core-1.0.3.jar
21/12/10 12:02:39 INFO Executor: Fetching file:///app/dependencies/ivy2/jars/org.antlr_antlr4-4.7.jar with timestamp 1639137758791
21/12/10 12:02:39 INFO Utils: /app/dependencies/ivy2/jars/org.antlr_antlr4-4.7.jar has been previously copied to /tmp/app/spark/spark-b4f75a4c-de12-4e0d-89fd-1bb64fc96015/userFiles-b97defce-65a3-4597-bd8d-92ff232a786e/org.antlr_antlr4-4.7.jar
21/12/10 12:02:39 INFO Executor: Fetching file:///app/dependencies/ivy2/ja

pySpark version: 3.1.1
Spark UI - http://309651fbdaab:4040


In [4]:
convert_to_standardized(RAW_DATA_DIR, STANDARDIZED_DATA_DIR)

2021/12/10 12:02:58 - root INFO: Start to process ./data/1.raw/ACM_ALERT_NOTES.csv
2021/12/10 12:03:04 - root INFO: Data saved to ./data/2.standardized/ACM_ALERT_NOTES.delta
2021/12/10 12:03:04 - root INFO: Time lapsed 6.01 s


2021/12/10 12:03:05 - root INFO: Start to process ./data/1.raw/ACM_MD_ALERT_STATUSES.csv
2021/12/10 12:03:07 - root INFO: Data saved to ./data/2.standardized/ACM_MD_ALERT_STATUSES.delta
2021/12/10 12:03:07 - root INFO: Time lapsed 1.41 s


2021/12/10 12:03:08 - root INFO: Start to process ./data/1.raw/ACM_ITEM_STATUS_HISTORY.csv
2021/12/10 12:03:09 - root INFO: Data saved to ./data/2.standardized/ACM_ITEM_STATUS_HISTORY.delta
2021/12/10 12:03:09 - root INFO: Time lapsed 1.22 s


2021/12/10 12:03:10 - root INFO: Start to process ./data/1.raw/ALERTS.csv
21/12/10 12:03:10 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
2021/12/10 12:03:11 - root INFO: Data saved to ./data/2.standardized/ALERTS.delta
2021/12/10 12:03:11 - root INFO: Time lapsed 1.15 s


# Create cleansed data

In [6]:
from pipeline.preprocessors import convert_standardized_to_cleansed

In [7]:
convert_standardized_to_cleansed()

Dimension 3 98


21/12/10 12:03:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
21/12/10 12:03:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
21/12/10 12:03:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
ANTLR Tool version 4.7 used for code generation does not match the current runtime version 4.8ANTLR Tool version 4.7 used for code generation does not match the current runtime version 4.8

# Prepare agent inputs

# Load data

In [9]:
from pipeline.preprocessors import transform_cleansed_to_application
transform_cleansed_to_application()

2021/12/10 12:03:50 - root INFO: Agent: rba_agent, Input written to ./data/4.application/agent-input/party_type_agent_input.delta, elapsed time: 1.08s
2021/12/10 12:03:51 - root INFO: Agent: rba_agent, Input written to ./data/4.application/agent-input/name_agent_input.delta, elapsed time: 1.05s
2021/12/10 12:03:52 - root INFO: Agent: rba_agent, Input written to ./data/4.application/agent-input/dob_agent_input.delta, elapsed time: 1.03s
2021/12/10 12:03:53 - root INFO: Agent: rba_agent, Input written to ./data/4.application/agent-input/pob_agent_input.delta, elapsed time: 1.05s
2021/12/10 12:03:54 - root INFO: Agent: rba_agent, Input written to ./data/4.application/agent-input/gender_agent_input.delta, elapsed time: 1.02s
2021/12/10 12:03:56 - root INFO: Agent: rba_agent, Input written to ./data/4.application/agent-input/national_id_agent_input.delta, elapsed time: 1.06s
2021/12/10 12:03:57 - root INFO: Agent: rba_agent, Input written to ./data/4.application/agent-input/document_number_

__Detailed implementation__

It's rather easy to implement if the goal is just to produce the dataframe for agent to consume. Some interim data need to be created to serve the purpose of analytics.

There are 2 main categories of transformations.
1. Interface/config transformation: Activities on the agent input config/interface.
1. Data transformation: Activities on the data based on the config/interface.

Steps
1. Create the agent input config.
    1. Interface/config transformation. Define agent input template. Each agent's input is a dictionary with 4 key-value pairs.
    ```
    {
        'ap': [],
        'ap_aliases': [],
        'wl': [] ,
        'wl_aliases': []
    }
    ```

        - `ap`: The primary value(s) of alerted party's specific attribute, e.g, name, it could be from one or multiple columns.
        - `ap_aliases`: The aliases of alerted party's specific attribute, it could be from one or multiple columns.
        - So on and so forth for `wl` and `wl_aliases`.

    1. Interface/config transformation. Define the list of agents. __Each agent's name must end with `_agent`.__
    ```
    agent_list = [
        'name_agent',
        'gender_agent'
    ]
    ```

    1. Interface/config transformation. Config the agent input by specifying which column(s) should be treated as the input of which agent's which party's primary or aliase value(s). 
    ```
    {
        'name_agent': {
            'ap': ['record_name', 'short_name'],
            'ap_aliases': ['alternate_name'],
            'wl': ['name_hit'],
            'wl_aliases': []
        },
        'gender_agent': {'ap': ['record_gender'],
                         'ap_aliases': [],
                         'wl': ['additional_infos_gender'],
                         'wl_aliases': []
                        },
    }
    ```
    Certain concepts need to be defined here.
        1. `level-1-key`: The name of each agent, it's `name_agent` and `gender_agent`.
        1. `level-1-value`: The value of each agent's config, it's a dictionary, e.g, 
        ```
        {
            'ap': ['record_name', 'short_name'],
            'ap_aliases': ['alternate_name'],
            'wl': ['name_hit'],
            'wl_aliases': []
        }
        ```
        1. `level-2-key`: The key of each agent config's value, or rather the key of `level-1-value`. It's `ap`, `ap_aliases`, `wl` and `wl_aliases`.
        1. `level-2-value`: The list of column names, e.g, `['record_name', 'short_name']`.

1. Create the interim agent input config and data. The interface is standardized from here onwards.  In reality, the data format can be more complex, e.g, national IDs we need to consider both type and document number.
    1. Interface/config transformation. Prepend `level-1-key` to `level-2-key` so that `level-2-key` can be used as new column names to host the interim data for analytics and/or debugging activites. Take `name_agent` for example.
    ```
    {
        'name_agent': {
            'name_agent_ap': ['record_name', 'short_name'],
            'name_agent_ap_aliases': ['alternate_name'],
            'name_agent_wl': ['name_hit'],
            'name_agent_wl_aliases': []
        }
    }
    ```
    1. Data transformation. Merge the values from `level-2-value` columns to `level-2-key` column. Below table will be the result.
    
| uuid | record_name | short_name | alternate_name | wl_primary_name | name_hit |   name_agent_ap   | name_agent_ap_aliases | name_agent_wl | name_agent_wl_aliases |
| ---- | :---------: | :--------: | :------------: | :-------------: | :------: | :---------------: | :-------------------: | :-----------: | :-------------------: |
| 1234 |  Jim Green  |    J.G.    |      Jim       |   James Greg    |   J.G    | [Jim Green, J.G.] |          Jim          |      J.G      |         None          |

1. Create the final agent input config and data based on the standardized interface.
    1. Interface/config transformation. Now we have a consistent schema to create the 1 list of alerted party values and 1 list of watchlist party values. We no longer need to worry about the customer specific schema, e.g, `record_name`, `short_name` and etcs. They have been standardized as `name_agent_ap`, `name_agent_ap_aliases` and etcs.
    ```
    {
        'name_agent': {'ap_all_names_aggregated': ['name_agent_ap', 'name_agent_ap_aliases'],
                       'wl_all_names_aggregated': ['name_agent_wl', 'name_agent_wl_aliases']
                      }
    }
    ```
    1. Data transformation. Merge the values from the primary and alias columns. Below table will be the result.
    
| uuid | ap_all_names_aggregated | wl_all_names_aggregated |
| ---- | :---------------------: | :---------------------: |
| 1234 | [Jim Green, J.G., Jim]  |          [J.G]          |